<a href="https://colab.research.google.com/github/DmitrijLeontev/Dmitrij_Leontev/blob/main/%D1%81%D1%82%D0%B0%D0%B6%D0%B8%D1%80%D0%BE%D0%B2%D0%BA%D0%B8%20/%D0%90%D0%B9%D0%A2%D0%B8%D0%9E%D0%BD_%D0%B7%D0%B0%D0%BC%D0%B5%D0%BD%D0%B0_%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%BE%D0%B2%D1%8B%D1%85_%D1%84%D0%B8%D0%BB%D1%8C%D1%82%D1%80%D0%BE%D0%B2/%D0%A0%D0%B0%D1%81%D0%BF%D0%BE%D0%B7%D0%BD%D0%B0%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_Whisper_Speech_Recognition_%D0%A2%D0%B8%D0%BD%D1%8C%D0%BA%D0%BE%D1%84%D1%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Монтируем Googl Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#установка необходимых библиотек
!pip install pydub
!pip install noisereduce
!pip install SpeechRecognition
!pip install git+https://github.com/openai/whisper.git
!pip3 install vosk
!pip install jiwer

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-hq5kv2vi
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-hq5kv2vi
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


Распознавание параметров аудио

In [ ]:
from pydub import AudioSegment

# чтение из файла любого формата проверяем первый файл
music = AudioSegment.from_file(file='/content/drive/MyDrive/Colab_Notebooks/it_on/dataset_wav/audio_2023-11-20_18-10-59.wav',
                               format='wav')

# продолжительность аудио (секунды)
print(music.duration_seconds)

# частота дискретизаци
print(music.frame_rate)

# количество каналов
print(music.channels)

13.7335
48000
1


### Обрезка аудио

In [ ]:
# чтение из файла любого формата
Slushai = AudioSegment.from_file(file='/content/drive/MyDrive/Colab_Notebooks/it_on/dataset_wav/audio_2023-11-20_18-10-59.wav',
                               format='wav')

# выведем параметры
print(Slushai.duration_seconds)
print(Slushai.frame_rate)
print(Slushai.channels)

# представление 40 секунд в миллисекундах
FORTY_SECONDS = 40 * 1000

# обрезка файла при помощи индексации
# возьмем первые 40 секунд аудиозаписи
Slushai[:FORTY_SECONDS]

13.7335
48000
1


Файл первый попался чистый без шума, на перваый взгляд.

In [ ]:
# сохраним фрагмент в файл
Slushai[:].export('part.wav', format='wav')

<_io.BufferedRandom name='part.wav'>

In [ ]:
#Переводим .ogg расширение в .wav,сделаем копию датасета  третьего блока из 25 файлов с расширением .wav

import os
from pydub import AudioSegment

new_folder = '/content/drive/MyDrive/Colab_Notebooks/it_on/dataset_wav'
org_folder = '/content/drive/MyDrive/Colab Notebooks/stagirovki/Recognition_Speech__To_Replace__Text_Filters/Голосовые_фильтры/Датасет_аудио/3'

# Проверяем существует ли папка new_folder, если нет - создаем ее
if not os.path.exists(new_folder):
    os.makedirs(new_folder)

for file_name in os.listdir(org_folder):
    if file_name.endswith(".ogg"):
        file_path = os.path.join(org_folder, file_name)
        audio_file = AudioSegment.from_ogg(file_path)
        new_file_name = os.path.splitext(file_name)[0] + ".wav"
        new_file_path = os.path.join(new_folder, new_file_name)
        audio_file.export(new_file_path, format="wav")


In [ ]:
# путь к папке с файлами
path = '/content/drive/MyDrive/Colab_Notebooks/it_on/dataset_wav'

Перевел в wav. сохранил.

Транскрибируем через Wosk и Whisper и Speech recognition

In [ ]:
#Основные импорты и загрузки
import pandas as pd
import noisereduce as nr
import soundfile as sf
import speech_recognition as sr
import whisper
import vosk
import wave
import json
from jiwer import wer
import librosa
import matplotlib.pyplot as plt

In [ ]:
# Создадим датафрейм для работы с текущим датасетом

file_names = [i for i in os.listdir(path)]
columns = ["file_names", "length",  "text_sr", "text_whisper", "text_tinkoff", "WER_sr", "target_text",  "WER_sr", "WER_whisp" ]


#создаём пустой датафрейм, где названия файлов выступают индексами строк
final_table = pd.DataFrame(columns=columns, index = range(0,len(file_names)))

d_bkp1 = final_table.copy()

final_table

,file_names,length,text_sr,text_whisper,text_tinkoff,WER_sr,target_text,WER_sr,WER_whisp
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
final_table['file_names'] = file_names

d_bkp2 = final_table.copy()

final_table

,file_names,length,text_sr,text_whisper,text_tinkoff,WER_sr,target_text,WER_sr,WER_whisp
0,audio_2023-11-20_18-10-59.wav,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,audio_2023-11-20_18-11-48.wav,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,audio_2023-11-20_18-12-00.wav,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,audio_2023-11-20_18-11-03.wav,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,audio_2023-11-20_18-12-04.wav,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,audio_2023-11-20_18-12-27.wav,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,audio_2023-11-20_18-12-09.wav,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,audio_2023-11-20_18-12-31.wav,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,audio_2023-11-20_18-12-38.wav,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,audio_2023-11-20_18-12-35.wav,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# сборка пути для открытия файла на транскрибацию по имени

recognizer = sr.Recognizer()

#проход по каждому индексу датафрейма

for i in range(0,final_table.shape[0]):
      # название файла из ячейки датафрейма по текущему индексу
      file_name = final_table.at[i, 'file_names']
      # полный путь к аудиофайлу для транскрибатора
      audio_file = os.path.join(path, file_name)
      with sr.AudioFile(audio_file) as source:
        audio = recognizer.record(source)
      # записываем результат транскрибации в переменную
      transcription = recognizer.recognize_google(audio, language="ru-RU")
      # записываем результат в соответствующий столбец по текущему индексу
      final_table.at[i, "text_sr"] = transcription

In [ ]:
final_table

,file_names,length,text_sr,text_whisper,text_tinkoff,WER_sr,target_text,WER_sr,WER_whisp
0,audio_2023-11-20_18-10-59.wav,NaN,Слушай сегодня поеду я из рисового в Орск наве...,NaN,NaN,NaN,NaN,NaN,NaN
1,audio_2023-11-20_18-11-48.wav,NaN,надо короче сгонять из Ясного в Гай посмотреть...,NaN,NaN,NaN,NaN,NaN,NaN
2,audio_2023-11-20_18-12-00.wav,NaN,Сегодня собираюсь ехать с другом из Оренбурга ...,NaN,NaN,NaN,NaN,NaN,NaN
3,audio_2023-11-20_18-11-03.wav,NaN,Слушай сегодня поеду я из рисового в Орск наве...,NaN,NaN,NaN,NaN,NaN,NaN
4,audio_2023-11-20_18-12-04.wav,NaN,сгоняли Мы короче с Оренбурга до Уфы на хатон ...,NaN,NaN,NaN,NaN,NaN,NaN
5,audio_2023-11-20_18-12-27.wav,NaN,на самом-то деле доехать из Челябинска в Екате...,NaN,NaN,NaN,NaN,NaN,NaN
6,audio_2023-11-20_18-12-09.wav,NaN,поехали мы короче с Бузулука до Магнитогорска ...,NaN,NaN,NaN,NaN,NaN,NaN
7,audio_2023-11-20_18-12-31.wav,NaN,Да разок я ездил из Ижевска в Пермь в целом ни...,NaN,NaN,NaN,NaN,NaN,NaN
8,audio_2023-11-20_18-12-38.wav,NaN,У меня знакомый кстати из Ярославля сам гонял ...,NaN,NaN,NaN,NaN,NaN,NaN
9,audio_2023-11-20_18-12-35.wav,NaN,У меня друг есть дело из Казани в Нижний Новго...,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
b_bkp3 = final_table.copy()

In [ ]:
excel_file = '/content/drive/MyDrive/Colab_Notebooks/it_on/final_table.xlsx'

# сохраняем датафрейм в эксель. Всё читается.
final_table.to_excel(excel_file, index=False)

In [ ]:
# попробуем использовать модель виспер
# создание модели, проход по нескольким строкам с попыткой транскрибации

model_whisper = whisper.load_model("large")

for i in range(0,final_table.shape[0]):

      file_name = final_table.at[i, 'file_names']
      audio_file = os.path.join(path, file_name)
      result = model_whisper.transcribe(audio_file)

      final_table.at[i, "text_whisper"] = result["text"]

In [ ]:
final_table

,file_names,length,text_sr,text_whisper,text_tinkoff,WER_sr,target_text,WER_sr,WER_whisp
0,audio_2023-11-20_18-10-59.wav,NaN,Слушай сегодня поеду я из рисового в Орск наве...,"Слушай, сегодня поеду я из Рысаева в Орск. На...",NaN,NaN,NaN,NaN,NaN
1,audio_2023-11-20_18-11-48.wav,NaN,надо короче сгонять из Ясного в Гай посмотреть...,"Надо, короче, сгонять из Ясного в Гай, посмот...",NaN,NaN,NaN,NaN,NaN
2,audio_2023-11-20_18-12-00.wav,NaN,Сегодня собираюсь ехать с другом из Оренбурга ...,Сегодня собираюсь ехать с другом из Оренбурга...,NaN,NaN,NaN,NaN,NaN
3,audio_2023-11-20_18-11-03.wav,NaN,Слушай сегодня поеду я из рисового в Орск наве...,"Слушай, сегодня поеду я из Рысаева в Орск. На...",NaN,NaN,NaN,NaN,NaN
4,audio_2023-11-20_18-12-04.wav,NaN,сгоняли Мы короче с Оренбурга до Уфы на хатон ...,"Сгоняли мы, короче, с Оренбурга до Уфы на хак...",NaN,NaN,NaN,NaN,NaN
5,audio_2023-11-20_18-12-27.wav,NaN,на самом-то деле доехать из Челябинска в Екате...,На самом деле доехать из Челябинска в Екатери...,NaN,NaN,NaN,NaN,NaN
6,audio_2023-11-20_18-12-09.wav,NaN,поехали мы короче с Бузулука до Магнитогорска ...,"Поехали мы, короче, с Бузулука до Магнитогорс...",NaN,NaN,NaN,NaN,NaN
7,audio_2023-11-20_18-12-31.wav,NaN,Да разок я ездил из Ижевска в Пермь в целом ни...,"Да, разок я ездил из Ижевска в Перемь. В цело...",NaN,NaN,NaN,NaN,NaN
8,audio_2023-11-20_18-12-38.wav,NaN,У меня знакомый кстати из Ярославля сам гонял ...,"У меня знакомый, кстати, из Ярославля сам гон...",NaN,NaN,NaN,NaN,NaN
9,audio_2023-11-20_18-12-35.wav,NaN,У меня друг есть дело из Казани в Нижний Новго...,У меня друг ездил из Казани в Нижний Новгород...,NaN,NaN,NaN,NaN,NaN


In [ ]:
b_bkp4 = final_table.copy()

In [ ]:
excel_file = '/content/drive/MyDrive/Colab_Notebooks/it_on/final_table.xlsx'

# сохраняем датафрейм в эксель. Теперь там два варианта транскрибации
final_table.to_excel(excel_file, index=False)

Whisper и Speech Recognition показали себя хорошо. Все транскрибировалось отлично. Воск не зашел. Пробуем Tinkoff VoiceKit. Сразу может не получиться , но пробуем за несколько подходов можно чего нибудь добиться. Есть проблема : Tinkoff VoiceKit не работает потоково , предется подавать по одному файлу и транскрибировать каждый отдельно. Это долго и муторно. Надо думать как автоматизировать подачу файлов.:-)

Файлы переведены в wav и сохранены.
Пробуем Тинькофф

# Tinkoff VoiceKit

Для использования сервиса Tinkoff VoiceKit необходима предварительная регистрация на платформе https://software.tinkoff.ru/auth/login/

После регистрации на Вашем балансе будет 1000 рублей, на которые Вы можете безвозмездно использовать сервис.

После регистрации Вам необходимо создать и сохранить 2 ключа API-key и SECRET-key. API_KEY можно сгенерировать в личном кабинете в разделе VoiceKit в любой момент времени. SECRET_KEY генерируется автоматически только при получении первого API_KEY, потом SECRET_KEY будет недоступен, поэтому крайне ВАЖНО сразу его сохранить.

In [ ]:
# установка библиотек
!pip install tinkoff-voicekit-client protobuf==3.20.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 9.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 52.6 MB/s eta 0:00:00
     ━━

In [ ]:
import getpass

# передаем API_KEY
API_KEY = getpass.getpass("Tinkoff API Key:")

Tinkoff API Key:··········


In [ ]:
# передаем SECRET_KEY
SECRET_KEY = getpass.getpass("Tinkoff SECRET Key:")

Tinkoff SECRET Key:··········


## Метод **Recognize**

Используется для распознавания аудиофайлов разных форматов (mp3, wav, s16)

Для транскрибации текста, любой метод принимает словарь параметров:

    audio_config = {"encoding": "MPEG_AUDIO",  
                    "sample_rate_hertz": 16000,
                    "num_channels": 2}

    "encoding" - кодировка, может быть: 'LINEAR16', 'ALAW', 'MULAW', 'LINEAR32F', 'RAW_OPUS', 'MPEG_AUDIO';

    "sample_rate_hertz" - частота дискретизации записи;

    "num_channels" - количество каналов записи (1 или 2).


In [ ]:
from tinkoff_voicekit_client import ClientSTT
from pprint import pprint

# создаем клиент, передаем ключи
client = ClientSTT(API_KEY, SECRET_KEY)

# указываем параметры аудио
audio_config = {
    "encoding": "MPEG_AUDIO",
    "sample_rate_hertz": 48000,
    "num_channels": 1,
    }

# вызываем метод recognize
response = client.recognize("/content/drive/MyDrive/Colab_Notebooks/it_on/dataset_wav/audio_2023-11-20_18-11-48.wav",
                            audio_config)
pprint(response)

{'results': [{'alternatives': [{'confidence': -4.075178,
                                'transcript': '',
                                'words': []}],
              'channel': 0,
              'end_time': '0.210s',
              'start_time': '0s'}]}


## Метод **StreamingRecognize**

Используется для распознавания речи в реальном времени: телефонных звонков, голосовых ассистентов и т.п. Этот метод содержит много разных фильтров и опций.

In [ ]:
# установка библиотеки ffmpeg-python
!pip install ffmpeg-python

In [ ]:
# создаем клиент, передаем ключи
client = ClientSTT(API_KEY, SECRET_KEY)

# указываем параметры аудио
audio_config = {
    "encoding": "LINEAR16",
    "sample_rate_hertz": 48000,
    "num_channels": 1,
}

# указываем параметры аудио
stream_config = {"config": audio_config}

# вызываем метод streaming_recognize
with open("/content/drive/MyDrive/Colab_Notebooks/it_on/dataset_wav/audio_2023-11-20_18-11-03.wav", "rb") as source:
    responses = client.streaming_recognize(source, stream_config)
    for response in responses:
        pprint(response)

{'results': [{'is_final': True,
              'recognition_result': {'alternatives': [{'confidence': -4.3896313,
                                                       'transcript': '',
                                                       'words': []}],
                                     'channel': 0,
                                     'end_time': '4.200s',
                                     'start_time': '4.140s'},
              'stability': 0.0}]}
{'results': [{'is_final': True,
              'recognition_result': {'alternatives': [{'confidence': -4.108748,
                                                       'transcript': '',
                                                       'words': []}],
                                     'channel': 0,
                                     'end_time': '27.690s',
                                     'start_time': '4.200s'},
              'stability': 0.0}]}


Метод StreamingRecognize позволяет использовать дополнительные опции в словаре параметров. Вот некоторые из них:

    audio_config = {"max_alternatives": 1,  
                    "vad_config": {},
                    "enable_automatic_punctuation": False,
                    "profanity_filter": False,
                    "enable_denormalization": False}

    "max_alternatives" - количество альтернатив распознанного текста, [1,15];

    "vad_config" - настройки для обнаружения голосовой активности;

    "enable_automatic_punctuation" - пунктуация;

    "profanity_filter" - фильтр ненормативной лексики;

    "enable_denormalization" - преобразование текста в числовые данные (время, дата и тп);

    ""

### Добавляем несколько альтернативных вариантов транскрибации текста

In [ ]:
# создаем клиент, передаем ключи
client = ClientSTT(API_KEY, SECRET_KEY)

# указываем параметры аудио
audio_config = {
    "encoding": "LINEAR16",   # кодировка
    "sample_rate_hertz": 48000, # частота
    "num_channels": 1,          # количество каналов
    "max_alternatives": 3       # количество альтернатив
}

# указываем параметры аудио
stream_config = {"config": audio_config}

# вызываем метод streaming_recognize
with open("/content/drive/MyDrive/Colab_Notebooks/it_on/dataset_wav/audio_2023-11-20_18-11-48.wav", "rb") as source:
    responses = client.streaming_recognize(source, stream_config)
    for response in responses:
        pprint(response)

{'results': [{'is_final': True,
              'recognition_result': {'alternatives': [{'confidence': -7.2897215,
                                                       'transcript': 'а',
                                                       'words': [{'confidence': 0.0,
                                                                  'end_time': '9.990s',
                                                                  'start_time': '9.960s',
                                                                  'word': 'а'}]},
                                                      {'confidence': -7.7214336,
                                                       'transcript': '',
                                                       'words': []},
                                                      {'confidence': -7.978822,
                                                       'transcript': 'и',
                                                       'words': [{'confidence': 0.0,
      

### Добавляем vad_config

In [ ]:
# создаем клиент, передаем ключи
client = ClientSTT(API_KEY, SECRET_KEY)

# указываем параметры vad (voice activity detection)
vad_config = {"silence_duration_threshold": 3,
              "silence_prob_threshold": 0.5}

# указываем параметры аудио
audio_config = {
    "encoding": "LINEAR16",   # кодировка
    "sample_rate_hertz": 48000, # частота
    "num_channels": 1,          # количество каналов
    "vad_config": vad_config    # vad параметры
}

# указываем параметры аудио
stream_config = {"config": audio_config}

# вызываем метод streaming_recognize
with open("/content/drive/MyDrive/Colab_Notebooks/it_on/dataset_wav/audio_2023-11-20_18-11-48.wav", "rb") as source:
    responses = client.streaming_recognize(source, stream_config)
    for response in responses:
        pprint(response)

{'results': [{'is_final': True,
              'recognition_result': {'alternatives': [{'confidence': -7.2878466,
                                                       'transcript': 'а',
                                                       'words': [{'confidence': 0.0,
                                                                  'end_time': '9.990s',
                                                                  'start_time': '9.960s',
                                                                  'word': 'а'}]}],
                                     'channel': 0,
                                     'end_time': '10.230s',
                                     'start_time': '9.570s'},
              'stability': 0.0}]}


### Включаем пунктуацию

In [ ]:
# создаем клиент, передаем ключи
client = ClientSTT(API_KEY, SECRET_KEY)

# указываем параметры аудио
audio_config = {
    "encoding": "MPEG_AUDIO",             # кодировка
    "sample_rate_hertz": 48000,           # частота
    "num_channels": 1,                    # количество каналов
    "enable_automatic_punctuation": True  # включаем пунктуацию
}

# указываем параметры аудио
stream_config = {"config": audio_config}

# вызываем метод streaming_recognize
with open("/content/drive/MyDrive/Colab_Notebooks/it_on/dataset_wav/audio_2023-11-20_18-11-48.wav", "rb") as source:
    responses = client.streaming_recognize(source, stream_config)
    for response in responses:
        pprint(response)

{'results': [{'is_final': True,
              'recognition_result': {'alternatives': [{'confidence': -4.075178,
                                                       'transcript': '',
                                                       'words': []}],
                                     'channel': 0,
                                     'end_time': '0.210s',
                                     'start_time': '0s'},
              'stability': 0.0}]}


Не получилось, надо дорабатывать, еще вернусь.
Вроде все правильно, но не транскрибирует.